# Criação do Modelo Estrela - Data Warehouse Financeiro

Este notebook cria as tabelas do modelo estrela para análise de cotações de ativos financeiros.

## Imports

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import os
import shutil

## Start Spark Session

In [ ]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Criação do Modelo Estrela - DW Financeiro") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")

## Define Delta Table Paths

In [ ]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Dimensões
delta_path_dim_ativo = f"{base_path}/dim_ativo"
delta_path_dim_tempo = f"{base_path}/dim_tempo"
delta_path_dim_cliente = f"{base_path}/dim_cliente"

# Fato
delta_path_fato_cotacao = f"{base_path}/fato_cotacao"

## Create Dimension Tables

In [ ]:
# Remove diretórios existentes
for path in [delta_path_dim_ativo, delta_path_dim_tempo, delta_path_dim_cliente]:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"✅ Diretório removido: {path}")

# Criar dimensão Ativo
spark.sql(f"""
CREATE TABLE IF NOT EXISTS dim_ativo (
    sk_ativo BIGINT COMMENT 'Chave surrogate da dimensão',
    cd_ativo STRING COMMENT 'Código de negociação do ativo',
    tp_mercado STRING COMMENT 'Tipo de mercado',
    nm_empresa STRING COMMENT 'Nome da empresa',
    ds_ativo STRING COMMENT 'Descrição do ativo',
    cd_isin STRING COMMENT 'Código ISIN do ativo',
    tp_ativo STRING COMMENT 'Tipo do ativo (ação, FII, etc)',
    nr_versao INT COMMENT 'Número da versão do registro',
    dt_inicio DATE COMMENT 'Data de início de validade do registro',
    dt_fim DATE COMMENT 'Data de fim de validade do registro',
    fl_corrente BOOLEAN COMMENT 'Flag que indica se é o registro corrente',
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização'
)
USING DELTA
COMMENT 'Dimensão de ativos financeiros'
LOCATION '{delta_path_dim_ativo}'
""")

print("✅ Tabela dim_ativo criada com sucesso!")

# Criar dimensão Tempo
spark.sql(f"""
CREATE TABLE IF NOT EXISTS dim_tempo (
    sk_tempo BIGINT COMMENT 'Chave surrogate da dimensão',
    dt_referencia DATE COMMENT 'Data de referência',
    nr_ano INT COMMENT 'Número do ano',
    nr_semestre INT COMMENT 'Número do semestre no ano (1-2)',
    nr_trimestre INT COMMENT 'Número do trimestre no ano (1-4)',
    nr_mes INT COMMENT 'Número do mês no ano (1-12)',
    nm_mes STRING COMMENT 'Nome do mês',
    nr_semana INT COMMENT 'Número da semana no ano',
    nr_dia INT COMMENT 'Número do dia no mês',
    nr_dia_ano INT COMMENT 'Número do dia no ano',
    nr_dia_semana INT COMMENT 'Número do dia na semana',
    nm_dia_semana STRING COMMENT 'Nome do dia da semana',
    fl_feriado BOOLEAN COMMENT 'Flag que indica se é feriado',
    fl_fim_semana BOOLEAN COMMENT 'Flag que indica se é fim de semana',
    fl_dia_util BOOLEAN COMMENT 'Flag que indica se é dia útil'
)
USING DELTA
COMMENT 'Dimensão de tempo'
LOCATION '{delta_path_dim_tempo}'
""")

print("✅ Tabela dim_tempo criada com sucesso!")

# Criar dimensão Cliente
spark.sql(f"""
CREATE TABLE IF NOT EXISTS dim_cliente (
    sk_cliente BIGINT COMMENT 'Chave surrogate da dimensão',
    cd_cliente STRING COMMENT 'Código do cliente',
    nm_cliente STRING COMMENT 'Nome do cliente',
    dt_nascimento DATE COMMENT 'Data de nascimento',
    tx_email STRING COMMENT 'Endereço de email',
    nr_telefone STRING COMMENT 'Número de telefone',
    nr_versao INT COMMENT 'Número da versão do registro',
    dt_inicio DATE COMMENT 'Data de início de validade do registro',
    dt_fim DATE COMMENT 'Data de fim de validade do registro',
    fl_corrente BOOLEAN COMMENT 'Flag que indica se é o registro corrente',
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização'
)
USING DELTA
COMMENT 'Dimensão de clientes'
LOCATION '{delta_path_dim_cliente}'
""")

print("✅ Tabela dim_cliente criada com sucesso!")

## Create Fact Table

In [ ]:
# Remove diretório existente da tabela fato
if os.path.exists(delta_path_fato_cotacao):
    shutil.rmtree(delta_path_fato_cotacao)
    print(f"✅ Diretório removido: {delta_path_fato_cotacao}")

# Criar tabela fato de cotações
spark.sql(f"""
CREATE TABLE IF NOT EXISTS fato_cotacao (
    -- Chaves para dimensões
    sk_ativo BIGINT COMMENT 'Chave surrogate da dimensão ativo',
    sk_tempo BIGINT COMMENT 'Chave surrogate da dimensão tempo',
    sk_cliente BIGINT COMMENT 'Chave surrogate da dimensão cliente',
    
    -- Métricas de preço
    vl_abertura DECIMAL(11,2) COMMENT 'Preço de abertura',
    vl_fechamento DECIMAL(11,2) COMMENT 'Preço de fechamento',
    vl_minimo DECIMAL(11,2) COMMENT 'Preço mínimo',
    vl_maximo DECIMAL(11,2) COMMENT 'Preço máximo',
    vl_medio DECIMAL(11,2) COMMENT 'Preço médio',
    
    -- Métricas de volume
    qt_negocios INT COMMENT 'Quantidade de negócios realizados',
    qt_titulos INT COMMENT 'Quantidade de títulos negociados',
    vl_volume DECIMAL(16,2) COMMENT 'Volume financeiro negociado',
    
    -- Métricas calculadas
    pc_variacao DECIMAL(7,4) COMMENT 'Percentual de variação no dia',
    
    -- Auditoria
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização',
    
    -- Constraints
    CONSTRAINT fk_dim_ativo FOREIGN KEY (sk_ativo) REFERENCES dim_ativo(sk_ativo),
    CONSTRAINT fk_dim_tempo FOREIGN KEY (sk_tempo) REFERENCES dim_tempo(sk_tempo),
    CONSTRAINT fk_dim_cliente FOREIGN KEY (sk_cliente) REFERENCES dim_cliente(sk_cliente)
)
USING DELTA
COMMENT 'Fato de cotações de ativos por cliente'
LOCATION '{delta_path_fato_cotacao}'
""")

print("✅ Tabela fato_cotacao criada com sucesso!")

## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()